# Deep learning

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [8]:
pickle_file = 'D:\\code_project\\datasource\\notMNIST\\notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 28, 28) (2000,)
Validation set (100, 28, 28) (100,)
Test set (100, 28, 28) (100,)


Reformat into a TensorFlow-friendly shape:

convolutions need the image data formatted as a cube (width by height by #channels)

labels as float 1-hot encodings.

In [11]:
import numpy as np
image_size = 28
num_labels = 10
num_channels = 1 #grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 28, 28, 1) (2000, 10)
Validation set (100, 28, 28, 1) (100, 10)
Test set (100, 28, 28, 1) (100, 10)


In [12]:
def accuracy(predictions, labels):
    return 100 * sum(np.argmax(predictions,1)==np.argmax(labels,1)) / labels.shape[0]

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # valiables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 *depth, num_hidden],stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels],stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    # model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        output = tf.matmul(hidden, layer4_weights) + layer4_biases
        return output
    
    # training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    # optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

W0926 11:02:28.515224  1048 deprecation.py:323] From <ipython-input-14-3d7bd5c71038>:39: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
num_steps = 101

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset : batch_data, tf_train_labels : batch_labels
        }
        _, batch_lo, batch_predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%50 == 0):
            print('Minibatch loss at step %d: %f' %(step, batch_lo))
            print('Minibatch training accuracy: %.1f%%' % accuracy(batch_predictions, batch_labels))
            print('Validation accuracy: %.1f%%' %accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' %accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 3.328694
Minibatch training accuracy: 6.2%
Validation accuracy: 11.0%
Minibatch loss at step 50: 1.116899
Minibatch training accuracy: 68.8%
Validation accuracy: 67.0%
Minibatch loss at step 100: 0.549629
Minibatch training accuracy: 87.5%
Validation accuracy: 81.0%
Test accuracy: 77.0%


# Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

# Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.